In [0]:
from pyspark.sql.functions import *

# Read claims files from bronze
claims_df = spark.read.parquet(
    "abfss://bronze@databricksdevetl.dfs.core.windows.net/claims/")

display(claims_df)

claims_df.createOrReplaceTempView("claims")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW quality_checks AS
SELECT 
  CONCAT(ClaimID,'-', datasource) AS ClaimID,
  ClaimID AS  SRC_ClaimID,
  TransactionID,
  PatientID,
  EncounterID,
  ProviderID,
  DeptID,
  cast(ServiceDate as date) ServiceDate,
  cast(ClaimDate as date) ClaimDate,
  PayorID,
  round(cast(ClaimAmount as double), 2) ClaimAmount,
  round(cast(PaidAmount as double), 2) PaidAmount,
  ClaimStatus,
  PayorType,
  round(cast(Deductible as double), 2) Deductible,
  round(cast(Coinsurance as double), 2) Coinsurance,
  round(cast(Copay as double), 2) Copay,
  cast(InsertDate as date) as SRC_InsertDate,
  cast(ModifiedDate as date) as SRC_ModifiedDate,
  datasource,
    CASE 
        WHEN ClaimID IS NULL OR TransactionID IS NULL OR PatientID IS NULL or ServiceDate IS NULL THEN TRUE
        ELSE FALSE
    END AS is_quarantined
FROM claims

In [0]:
%sql
SELECT * FROM quality_checks;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_dev.silver.claims (
ClaimID string,
SRC_ClaimID string,
TransactionID string,
PatientID string,
EncounterID string,
ProviderID string,
DeptID string,
ServiceDate date,
ClaimDate date,
PayorID string,
ClaimAmount double,
PaidAmount double,
ClaimStatus string,
PayorType string,
Deductible double,
Coinsurance double,
Copay double,
SRC_InsertDate date,
SRC_ModifiedDate date,
datasource string,
is_quarantined boolean,
audit_insertdate timestamp,
audit_modifieddate timestamp,
is_current boolean
)
USING DELTA;

In [0]:
%sql
-- Update old record to implement SCD Type 2
MERGE INTO databricks_dev.silver.claims AS target
USING quality_checks AS source
ON target.ClaimID = source.ClaimID 
AND target.is_current = true
WHEN MATCHED AND (
    target.SRC_ClaimID != source.SRC_ClaimID OR
    target.TransactionID != source.TransactionID OR
    target.PatientID != source.PatientID OR
    target.EncounterID != source.EncounterID OR
    target.ProviderID != source.ProviderID OR
    target.DeptID != source.DeptID OR
    target.ServiceDate != source.ServiceDate OR
    target.ClaimDate != source.ClaimDate OR
    target.PayorID != source.PayorID OR
    target.ClaimAmount != source.ClaimAmount OR
    target.PaidAmount != source.PaidAmount OR
    target.ClaimStatus != source.ClaimStatus OR
    target.PayorType != source.PayorType OR
    target.Deductible != source.Deductible OR
    target.Coinsurance != source.Coinsurance OR
    target.Copay != source.Copay OR
    target.SRC_InsertDate != source.SRC_InsertDate OR
    target.SRC_ModifiedDate != source.SRC_ModifiedDate OR
    target.datasource != source.datasource OR
    target.is_quarantined != source.is_quarantined
) THEN
  UPDATE SET
    target.is_current = false,
    target.audit_modifieddate = current_timestamp()

In [0]:
%sql
-- Insert new record to implement SCD Type 2
MERGE INTO databricks_dev.silver.claims AS target
USING quality_checks AS source
ON target.ClaimID = source.ClaimID 
AND target.is_current = true
WHEN NOT MATCHED THEN
  INSERT (
    ClaimID,
    SRC_ClaimID,
    TransactionID,
    PatientID,
    EncounterID,
    ProviderID,
    DeptID,
    ServiceDate,
    ClaimDate,
    PayorID,
    ClaimAmount,
    PaidAmount,
    ClaimStatus,
    PayorType,
    Deductible,
    Coinsurance,
    Copay,
    SRC_InsertDate,
    SRC_ModifiedDate,
    datasource,
    is_quarantined,
    audit_insertdate,
    audit_modifieddate,
    is_current
  )
  VALUES (
    source.ClaimID,
    source.SRC_ClaimID,
    source.TransactionID,
    source.PatientID,
    source.EncounterID,
    source.ProviderID,
    source.DeptID,
    source.ServiceDate,
    source.ClaimDate,
    source.PayorID,
    source.ClaimAmount,
    source.PaidAmount,
    source.ClaimStatus,
    source.PayorType,
    source.Deductible,
    source.Coinsurance,
    source.Copay,
    source.SRC_InsertDate,
    source.SRC_ModifiedDate,
    source.datasource,
    source.is_quarantined,
    current_timestamp(),
    current_timestamp(),
    true
  );

In [0]:
%sql
SELECT * FROM databricks_dev.silver.claims;